Chapitre 7 - Du script à l'application - module, gestion des erreurs et recherche SQL
===

Notre application et nos compétences ont sérieusement grossies : nous dommes désormais capables de traiter du JSON, du CSV, de récupérer des informations depuis le web mais aussi de créer notre propre application web, bien qu'elle reste simple pour le moment.

Cette dite application web contient désormais 39 lignes de Python pour 2 pages et un type de ressource SQL. Vous vous rendrez vite compte que la manipulation de fichiers comme celui-ci rend les choses complexes. De fait, on préfère en général divisé les fichiers pythons comme nous divisons les templates : en les incluant ou en faisant appel à eux quand nous en avons besoin.

## Les modules et packages

Ce découpage en python s'appelle la modularisation : il s'agit de créer des modules qui à terme forme des packages (groupe de modules) et ainsi pouvoir se retrouver dans des applications un peu plus large que 39 lignes.

Nous avons déjà utilisés des modules. Ainsi quand on tapez

```python
from flask import Flask
```

Nous importions depuis le module principal du package `flask` la classe `Flask` qui nous permettait de créer notre application. Un module est en fait un fichier python. Ainsi, si je crée un fichier `modeles.py` pour y ranger l'ensemble de mes modèles de données SQL (et pouvoir m'y retrouver facilement), je pourrai à terme faire `from modeles import Place` !

Les modules peuvent comprendre tout ce que python peut faire, à savoir : 

- des variables
- des fonctions
- des classes

### Créer un package, c'est simple !

La création de package est en fait extrêment simple : 

0. On crée un fichier `run.py` qui nous permettra d'utiliser ce qui est dans gazetteer
1. On crée un dossier : `gazetteer/` par exemple.
2. On ajoute dans ce dossier un fichier `__init__.py`
3. On écrit dans ce fichier `__init__.py`.
4. On écrit dans ce fichier `ma_variable = 0`

Les fichiers `__init__.py` sont nécessaire pour que ce dossier soit compris comme package. Le résultat serait donc

```
gazetteer/
   |- __init__.py
run.py
```

Avec `__init__.py` qui comprend le code suivant :

In [1]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-01/gazetteer/__init__.py

Cela permettra à `run.py` d'utiliser la variable en faisant :

In [3]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-01/run.py

Et le résultat de son appel sera :

In [15]:
%run cours-packages/exemple-01/run.py

0


#### Pour lancer l'exemple

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-01
python run.py
```

#### Résumé d'un module simple avec dossier

- Un module simple avec dossier comprenant un `__init__.py` est appelé ou importé en utilisant le nom du dossier. 
- On peut y stocker des variables, classes et fonctions.

### Un package, des modules

Il est possible pour un package de comporter plusieurs modules, ainsi pour notre Gazetteer il est possible de diviser par exemple les modeles des routes. Pour le moment, créons cet ensemble :

0. On crée un fichier `run.py` qui nous permettra d'utiliser ce qui est dans gazetteer
1. On crée un dossier : `gazetteer/` par exemple.
2. On ajoute dans ce dossier un fichier `__init__.py`
3. On crée un fichier `application.py`
4. On écrit dans ce fichier `mon_module = "application"`
5. On crée un fichier `modeles.py`
6. On écrit dans ce fichier `mon_module = "modeles"`

```
gazetteer/
   |- __init__.py
   |- application.py
   |- modeles.py
run.py
```

In [10]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/gazetteer/application.py

In [11]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/gazetteer/modeles.py

Cela permettra à `run.py` d'utiliser la variable en important les modules du packages. On importe des modules et sous-modules en rétablissant le chemin via des `.` :

```python
from gazetteer.application import mon_module
```

Ainsi, on écrira dans notre `run` :

In [27]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/run.py

Et le résultat de son appel sera :

In [28]:
# Cette commande permet de lancer le script directement depuis ce module
! cd cours-packages/exemple-02 && python run.py

modeles


#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-02
python run.py
```

#### Problème ?

Nous rencontrons un premier problème : `mon_module` étant importé depuis deux modules (`gazetter.application` et `gazetteer.modeles`), les deux rentrent en conflit et la dernière prend le dessus. Nous avons plusieurs solutions à ce problème 

1. `from gazetteer import application` et `print(application.mon_module)` qui permet de garder le nom du sous-module
2. `import gazetteer.application` et `print(gazetteer.application.mon_module)` : long à écrire mais impossible d'avoir des conflits et compréhensible
3. `from gazetteer.application import mon_module as autre_nom` : on renomme, on perd l'information d'origine mais on reste sur un nom court.

Pour continuer l'exemple, on écrit un fichier `run2.py` qui reprend cette syntaxe :

In [29]:
# Raccourci pour afficher le contenu de l'exemple
%pycat cours-packages/exemple-02/run2.py

Et le résultat de son appel sera :

In [30]:
# Cette commande permet de lancer le script directement depuis ce module
! cd cours-packages/exemple-02 && python run2.py

application
modeles


#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-pages/exemple-02
python run2.py
```

#### Résumé d'un packages avec plusieurs modules

- Un package se construit avec un dossier comprenant un `__init__.py`
- Les fichiers qui le constituent sont importables comme modules (`from package import module`)
- Les imports de ressources dans des modules peuvent être construits de trois manière :
    - `import package.module`
        - Appelée via `package.module.ressource`
    - `from package import module`
        - Appelée via `module.ressource`
    - `from package.module import resource`
        - Appelée via `ressource`
- En cas de conflit, on peut:
    - changer de méthode d'import
    - renommer la ressource importée via `import ___ as ___` ou `from ___ import ___ as ____`

### De module à module : l'import relatif

### Notre application en packages

## Les erreurs

### Que faire quand on a une erreur

### Que faire quand on rencontre CETTE erreur

### Gestion des erreurs : afficher une page d'erreur quand un lieu n'existe pas